In [3]:
import json

# Charger le dataset
with open('data_modified_final.json', 'r') as f:
    data = json.load(f)

# Afficher quelques exemples pour vérifier
#for question in data['questions'][:3]:
#    print(question)


questions = data['questions']



In [4]:
import unicodedata

def normalize_string(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

# Prétraiter les questions pour enlever les accents
for question in questions:
    question['question'] = normalize_string(question['question'])
    question['options'] = [normalize_string(option) for option in question['options']]


In [5]:
########### il faut installer le module : pip install ortools  ############

from ortools.linear_solver import pywraplp

def generer_sujets_lp(nb_sujets, total_points, duree_totale, pourcentage_facile, pourcentage_moyen, pourcentage_difficile):
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Définir les variables de décision
    x = []
    for i in range(len(questions)):
        x.append(solver.BoolVar(f'x_{i}'))

    # Contraintes de somme des points
    solver.Add(sum(questions[i]['marks'] * x[i] for i in range(len(questions))) == total_points)

    # Contraintes de durée totale
    solver.Add(sum(int(questions[i]['duree_reponse'].split()[0]) * x[i] for i in range(len(questions))) == duree_totale)

    # Contraintes de niveau de difficulté
    solver.Add(sum(x[i] for i in range(len(questions)) if questions[i]['difficulte'] == 1) == pourcentage_facile * nb_sujets)
    solver.Add(sum(x[i] for i in range(len(questions)) if questions[i]['difficulte'] == 2) == pourcentage_moyen * nb_sujets)
    solver.Add(sum(x[i] for i in range(len(questions)) if questions[i]['difficulte'] == 3) == pourcentage_difficile * nb_sujets)

    # Objectif : Maximiser la sélection des questions
    solver.Maximize(sum(x[i] for i in range(len(questions))))

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        sujets = []
        for i in range(nb_sujets):
            sujet = []
            for j in range(len(questions)):
                if x[j].solution_value() > 0:
                    sujet.append(questions[j])
            sujets.append(sujet)
        return sujets
    else:
        print('Aucune solution optimale trouvée')
        return None

# Appel de la fonction
sujets_generes = generer_sujets_lp(4, 20, 45, 0.3, 0.5, 0.2)


ModuleNotFoundError: No module named 'ortools'

In [ ]:
# ############# new ############

# try:
#     from ortools.linear_solver import pywraplp
# except ImportError:
#     import sys
#     !{sys.executable} -m pip install ortools
#     from ortools.linear_solver import pywraplp

# def generer_sujets_lp(nb_sujets, total_points, duree_totale, pourcentage_facile, pourcentage_moyen, pourcentage_difficile):
#     solver = pywraplp.Solver.CreateSolver('SCIP')

#     # Définir les variables de décision
#     x = []
#     for i in range(len(questions)):
#         x.append(solver.BoolVar(f'x_{i}'))

#     # Contraintes de somme des points
#     solver.Add(sum(questions[i]['marks'] * x[i] for i in range(len(questions))) == total_points)

#     # Contraintes de durée totale
#     solver.Add(sum(int(questions[i]['duree_reponse'].split()[0]) * x[i] for i in range(len(questions))) == duree_totale)

#     # Contraintes de niveau de difficulté
#     solver.Add(sum(x[i] for i in range(len(questions)) if questions[i]['difficulte'] == 1) == pourcentage_facile * nb_sujets)
#     solver.Add(sum(x[i] for i in range(len(questions)) if questions[i]['difficulte'] == 2) == pourcentage_moyen * nb_sujets)
#     solver.Add(sum(x[i] for i in range(len(questions)) if questions[i]['difficulte'] == 3) == pourcentage_difficile * nb_sujets)

#     # Objectif : Maximiser la sélection des questions
#     solver.Maximize(sum(x[i] for i in range(len(questions))))

#     status = solver.Solve()

#     if status == pywraplp.Solver.OPTIMAL:
#         sujets = []
#         for i in range(nb_sujets):
#             sujet = []
#             for j in range(len(questions)):
#                 if x[j].solution_value() > 0:
#                     sujet.append(questions[j])
#             sujets.append(sujet)
#         return sujets
#     else:
#         print('Aucune solution optimale trouvée')
#         return None

# # Appel de la fonction
# sujets_generes = generer_sujets_lp(4, 20, 45, 0.3, 0.5, 0.2)


In [ ]:
######################## Nous utilisons un MLP deja pre entrainer #################


from sklearn.neural_network import MLPClassifier
import numpy as np

# Préparer les données pour l'entraînement
X = []
y = []

for question in questions:
    X.append([question['difficulte'], int(question['duree_reponse'].split()[0]), question['marks']])
    y.append(1)  # Toutes les questions sont valides

X = np.array(X)
y = np.array(y)

# Entraîner le modèle
clf = MLPClassifier(random_state=1, max_iter=300).fit(X, y)

def generer_sujets_nn(nb_sujets, total_points, duree_totale, pourcentage_facile, pourcentage_moyen, pourcentage_difficile):
    sujets = []
    for _ in range(nb_sujets):
        sujet = []
        points_restants = total_points
        duree_restante = duree_totale
        while points_restants > 0 and duree_restante > 0:
            questions_valides = [q for q in questions if q['marks'] <= points_restants and int(q['duree_reponse'].split()[0]) <= duree_restante]
            if not questions_valides:
                break
            X_valides = np.array([[q['difficulte'], int(q['duree_reponse'].split()[0]), q['marks']] for q in questions_valides])
            y_pred = clf.predict(X_valides)
            question_choisie = questions_valides[np.argmax(y_pred)]
            sujet.append(question_choisie)
            points_restants -= question_choisie['marks']
            duree_restante -= int(question_choisie['duree_reponse'].split()[0])
        sujets.append(sujet)
    return sujets

# Appel de la fonction
sujets_generes_nn = generer_sujets_nn(4, 20, 45, 0.3, 0.5, 0.2)


In [ ]:
################### Enf in nous genereons des sujet ###################


def ecrire_sujets(sujets, filename='sujets_generes.txt'):
    with open(filename, 'w') as file:
        for i, sujet in enumerate(sujets):
            file.write(f'Sujet {i + 1}:\n')
            for question in sujet:
                options = ', '.join([f'"{opt}"' for opt in question['options']])
                file.write(f"{question['question']} {options}, {question['marks']} points, {question['duree_reponse']}, difficulté {question['difficulte']}\n")
            file.write('\n')

# Écrire les sujets générés
ecrire_sujets(sujets_generes_nn)


In [ ]:
"""Explications Mathématiques et Optimisation
Programmation Linéaire:

Utilisation des contraintes linéaires pour s'assurer que les sujets générés respectent les critères de points, de durée, et de niveaux de difficulté.
Solution optimale trouvée en maximisant la sélection de questions tout en respectant les contraintes définies.
Réseaux de Neurones:

Entraînement d'un réseau de neurones sur les questions disponibles pour apprendre à sélectionner des questions valides.
Utilisation du modèle pour générer des sujets équilibrés.
Optimisation:

Les algorithmes d'optimisation sont utilisés pour garantir que les sujets générés respectent les contraintes et sont équilibrés en termes de points, de durée, et de niveaux de difficulté.
Avec cette approche, vous pouvez générer des sujets d'examen respectant les contraintes définies et justifier les choix méthodologiques auprès du jury en utilisant des concepts de programmation linéaire et d'intelligence artificielle."""